In [9]:
import os
import fiona
import pandas as pd
import geopandas as gpd
from shapely.wkt import loads

# BRO Soil map dataset

--> This dataset is a multi-layered soil database that includes pedological properties and land characteristics.

## Load data

In [2]:
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
path = os.path.join(current_dir, "../data/raw/Wageningen_soil_map/BRO-SGM-DownloadBodemkaart-V2024-01_1.gpkg")

In [5]:
layers = fiona.listlayers(path)
layers

['areaofpedologicalinterest',
 'soilarea',
 'nga_properties',
 'soilmap',
 'normalsoilprofiles',
 'normalsoilprofiles_landuse',
 'soilhorizon',
 'soilhorizon_fractionparticlesize',
 'soillayer',
 'soil_units',
 'soilcharacteristics_bottomlayer',
 'soilcharacteristics_toplayer',
 'soilarea_normalsoilprofile',
 'soilarea_soilunit',
 'soilarea_soilunit_soilcharacteristicstoplayer',
 'soilarea_soilunit_soilcharacteristicsbottomlayer']

## Load needed layers

In [ ]:
# NOTE: "normalsoilprofile" cannot be joined to any table as it does not have id number. Lower and Upper values are constant (0, 1.2) so we dont need this table at all. Work only with "soilhorizon"!

In [11]:
soilarea = gpd.read_file(path, layer="soilarea")  # table with geo
soilarea_normalsoilprofile = gpd.read_file(path, layer="soilarea_normalsoilprofile") # relationship between soilarea and horizon
horizon = gpd.read_file(path, layer="soilhorizon")   # table with soil characteristics

/Users/Administrator/Documents/University/Year 3/2b/Thesis/Bachelor-Thesis/spatial_env/lib/python3.9/site-packages/pyogrio/raw.py:198: RuntimeWarning: Unexpected data type for record 1 in column validfrom
  return ogr_read(


## Merge data

In [12]:
# join 'soilhorizon' with 'soilarea_normalsoilprofile'
horizon_profiles_area = horizon.merge(soilarea_normalsoilprofile, on="normalsoilprofile_id")
area_geometry = soilarea[["maparea_id", "geometry"]]    # here we keep only necessary columns
horizon_with_geom = horizon_profiles_area.merge(area_geometry, on="maparea_id")

--> In one polygon we have values for five levels of soil

In [28]:
horizon_with_geom = horizon_with_geom[["layernumber", "organicmattercontent", "lutitecontent", "geometry"]].copy()
horizon_with_geom

,layernumber,organicmattercontent,lutitecontent,geometry
0,1,6.8,2,"POLYGON ((226756 475715, 226757 475683, 226757..."
1,1,6.8,2,"POLYGON ((232421 476085, 232441 476105, 232444..."
2,1,6.8,2,"POLYGON ((231954 476332, 231936 476361, 231898..."
3,1,6.8,2,"POLYGON ((232281 476788, 232330 476778, 232401..."
4,1,6.8,2,"POLYGON ((228217 476637, 228184 476646, 228151..."
...,...,...,...,...
207547,4,3.0,65,"POLYGON ((127413.821 475025.001, 127398.475 47..."
207548,4,3.0,65,"POLYGON ((170875.765 441757.232, 170899.975 44..."
207549,4,3.0,65,"POLYGON ((129167.816 436603.082, 129187.925 43..."
207550,4,3.0,65,"POLYGON ((137509.228 438430.345, 137485.162 43..."


In [ ]:
# NOTE: select 1st, 4th, 5th layer numbers! (most important in the key research paper)

In [13]:
polygon_str = "POLYGON ((232281.000000011 476788.000000007, 232330.000000034 476778.000000004, 232401.000000004 476782.999999996, 232469.000100002 476788.000000007, 232496.999900006 476798.000100013, 232524.999900006 476798.000100013, 232568.000000004 476788.000000007, 232624.000000015 476781.000000004, 232685.000000007 476782.999999996, 232745.999900024 476809.000100005, 232792.000000022 476848.999899995, 232815.000100002 476879.999999996, 232825.000000019 476925, 232825.000000019 476965.999899998, 232817.000000011 477008.999999996, 232817.000000011 477035.000000004, 232807.000100035 477060, 232794.000000004 477042.000000004, 232777.000000011 477045.000000004, 232683.000000026 477045.000000004, 232655 477030.000100009, 232628.999999996 477014.000000007, 232601 477002.000100013, 232579 476990.999899998, 232533.000100032 476990.999899998, 232495 476999.000100013, 232454 476999.000100013, 232418.999899995 476996.999899998, 232388.000100002 476989.000000007, 232362.999900024 476968.999899998, 232336.999999996 476936.000000007, 232311.999999996 476912.999900002, 232292.000000007 476897.000000004, 232273.999900006 476872.000000004, 232266 476854.000100009, 232259.000000011 476839.000000011, 232259.000000011 476824, 232264.000000019 476809.000100005, 232281.000000011 476788.000000007))"
target_geom = loads(polygon_str)

view = horizon_with_geom[horizon_with_geom["geometry"]==target_geom]

In [14]:
view

,normalsoilprofile_id,layernumber,faohorizonnotation,lowervalue,uppervalue,staringseriesblock,organicmattercontent,minimumorganicmattercontent,maximumorganicmattercontent,acidity,...,sandmedian,minimumsandmedian,maximumsandmedian,siltcontent,density,inspireid,validfrom,beginlifespanversion,maparea_id,geometry
3,4160,1,Ah,0.00,0.08,105,6.8,2.0,12.0,3.2,...,250,200,400,6,1.271,sgm.V2024-1.profile.4160.horizon.1,NaT,NaT,V2024-1..soilarea.0000013951,"POLYGON ((232281 476788, 232330 476778, 232401..."
213,4160,2,Eu,0.08,0.15,205,2.3,0.5,5.0,3.3,...,250,200,400,6,1.489,sgm.V2024-1.profile.4160.horizon.2,NaT,NaT,V2024-1..soilarea.0000013951,"POLYGON ((232281 476788, 232330 476778, 232401..."
423,4160,3,Bhs,0.15,0.30,205,4.7,1.0,12.0,4.3,...,250,200,400,6,1.397,sgm.V2024-1.profile.4160.horizon.3,NaT,NaT,V2024-1..soilarea.0000013951,"POLYGON ((232281 476788, 232330 476778, 232401..."
633,4160,4,BCy,0.30,0.60,205,0.9,0.3,2.0,4.5,...,250,200,400,4,1.551,sgm.V2024-1.profile.4160.horizon.4,NaT,NaT,V2024-1..soilarea.0000013951,"POLYGON ((232281 476788, 232330 476778, 232401..."
843,4160,5,Cy,0.60,1.20,205,0.3,0.1,1.0,4.8,...,250,200,400,4,1.578,sgm.V2024-1.profile.4160.horizon.5,NaT,NaT,V2024-1..soilarea.0000013951,"POLYGON ((232281 476788, 232330 476778, 232401..."
